In [5]:
import pandas as pd 
pay_bill = pd.read_csv('data/credit_record.csv')
pay_bill

,ID,MONTHS_BALANCE,STATUS
0,5001711,0,X
1,5001711,-1,0
2,5001711,-2,0
3,5001711,-3,0
4,5001712,0,C
...,...,...,...
1048570,5150487,-25,C
1048571,5150487,-26,C
1048572,5150487,-27,C
1048573,5150487,-28,C


In [6]:
def get_unused_user(df):
    df = df.groupby('ID').apply(lambda x: x['STATUS'].isin(['X']).all()).reset_index()
    df = df[df[0] == True]
    df.rename(columns={0: 'credit_decision'}, inplace=True)
    df['credit_decision'] = 0
    return df
unused_df = get_unused_user(pay_bill)
unused_df

,ID,credit_decision
2,5001713,0
3,5001714,0
4,5001715,0
16,5001731,0
17,5001732,0
...,...,...
45947,5150409,0
45948,5150410,0
45965,5150460,0
45979,5150481,0


In [7]:
credit_decision = pd.DataFrame()
used_df = pay_bill[pay_bill['STATUS'] != 'X']
credit_decision['ID'] = used_df['ID'].unique()

def get_used_credit_decision(id):
    '''Determine for a given df, if the client is good, bad or unused status according to the credit usage.'''
    client_status = used_df[used_df['ID'] == id]['STATUS'].tolist()
    c_count = client_status.count('C')
    total_count = len(client_status)
    if c_count / total_count >= 0.7:
        return 1
    else:
        return -1

credit_decision['credit_decision'] = credit_decision['ID'].map(get_used_credit_decision)
credit_decision

,ID,credit_decision
0,5001711,-1
1,5001712,-1
2,5001717,-1
3,5001718,-1
4,5001719,1
...,...,...
41444,5150480,1
41445,5150482,-1
41446,5150484,-1
41447,5150485,-1


In [8]:
credit_decision['credit_decision'].value_counts()

credit_decision
-1    29402
 1    12047
Name: count, dtype: int64

In [9]:
final_decision = pd.concat([unused_df, credit_decision])
final_decision

,ID,credit_decision
2,5001713,0
3,5001714,0
4,5001715,0
16,5001731,0
17,5001732,0
...,...,...
41444,5150480,1
41445,5150482,-1
41446,5150484,-1
41447,5150485,-1


In [11]:
# final_decision.to_csv('data/credit_decision.csv', index=False)

In [12]:
app_record = pd.read_csv('data/application_record.csv')
app_record = pd.merge(app_record, final_decision, on='ID', how='inner')
app_record

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,credit_decision
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0,1
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0,1
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0,-1
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0,-1
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36452,5149828,M,Y,Y,0,315000.0,Working,Secondary / secondary special,Married,House / apartment,-17348,-2420,1,0,0,0,Managers,2.0,-1
36453,5149834,F,N,Y,0,157500.0,Commercial associate,Higher education,Married,House / apartment,-12387,-1325,1,0,1,1,Medicine staff,2.0,-1
36454,5149838,F,N,Y,0,157500.0,Pensioner,Higher education,Married,House / apartment,-12387,-1325,1,0,1,1,Medicine staff,2.0,-1
36455,5150049,F,N,Y,0,283500.0,Working,Secondary / secondary special,Married,House / apartment,-17958,-655,1,0,0,0,Sales staff,2.0,-1


In [28]:
income_train = pd.read_csv('data/income_train.csv')
income_test = pd.read_csv('data/income_test.csv')
income = pd.concat([income_train, income_test])
income.drop('income_>50K', axis=1, inplace=True)
income

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country
0,67,Private,366425,Doctorate,16,Divorced,Exec-managerial,Not-in-family,White,Male,99999,0,60,United-States
1,17,Private,244602,12th,8,Never-married,Other-service,Own-child,White,Male,0,0,15,United-States
2,31,Private,174201,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,40,United-States
3,58,State-gov,110199,7th-8th,4,Married-civ-spouse,Transport-moving,Husband,White,Male,0,0,40,United-States
4,25,State-gov,149248,Some-college,10,Never-married,Other-service,Not-in-family,Black,Male,0,0,40,United-States
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
894,27,Private,302406,Masters,14,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,55,United-States
895,17,Private,218124,11th,7,Never-married,Sales,Own-child,White,Female,0,0,20,United-States
896,24,Private,127139,Some-college,10,Never-married,Adm-clerical,Own-child,White,Female,0,0,40,United-States
897,35,Private,132879,Some-college,10,Married-civ-spouse,Tech-support,Husband,White,Male,0,0,40,United-States


In [29]:
def education_categorize(x):
    if x['education'] in ['9th','10th','11th','12th','HS-grad']:
        return 'Secondary / secondary special'
    elif x['education'] in ['1st-4th','5th-6th','7th-8th','Preschool']:
        return 'Lower secondary'
    elif x['education'] in ['Assoc-acdm','Assoc-voc','Some-college']:
        return 'Higher education'
    else:
        return 'Academic degree'
income['education'] = income.apply(education_categorize, axis=1)
income

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country
0,67,Private,366425,Academic degree,16,Divorced,Exec-managerial,Not-in-family,White,Male,99999,0,60,United-States
1,17,Private,244602,Secondary / secondary special,8,Never-married,Other-service,Own-child,White,Male,0,0,15,United-States
2,31,Private,174201,Academic degree,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,40,United-States
3,58,State-gov,110199,Lower secondary,4,Married-civ-spouse,Transport-moving,Husband,White,Male,0,0,40,United-States
4,25,State-gov,149248,Higher education,10,Never-married,Other-service,Not-in-family,Black,Male,0,0,40,United-States
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
894,27,Private,302406,Academic degree,14,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,55,United-States
895,17,Private,218124,Secondary / secondary special,7,Never-married,Sales,Own-child,White,Female,0,0,20,United-States
896,24,Private,127139,Higher education,10,Never-married,Adm-clerical,Own-child,White,Female,0,0,40,United-States
897,35,Private,132879,Higher education,10,Married-civ-spouse,Tech-support,Husband,White,Male,0,0,40,United-States


In [30]:
income['education'].value_counts()

education
Secondary / secondary special    18749
Higher education                 13332
Academic degree                  11123
Lower secondary                   1652
Name: count, dtype: int64

In [31]:
income.info()

<class 'pandas.core.frame.DataFrame'>
Index: 44856 entries, 0 to 898
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   age              44856 non-null  int64 
 1   workclass        42358 non-null  object
 2   fnlwgt           44856 non-null  int64 
 3   education        44856 non-null  object
 4   educational-num  44856 non-null  int64 
 5   marital-status   44856 non-null  object
 6   occupation       42350 non-null  object
 7   relationship     44856 non-null  object
 8   race             44856 non-null  object
 9   gender           44856 non-null  object
 10  capital-gain     44856 non-null  int64 
 11  capital-loss     44856 non-null  int64 
 12  hours-per-week   44856 non-null  int64 
 13  native-country   44093 non-null  object
dtypes: int64(6), object(8)
memory usage: 5.1+ MB


In [32]:
import numpy as np
from scipy import stats 
def income_agg(x):
    if x.dtypes == np.int64:
        return x.mean()
    elif x.dtypes == object:
        return x.mode()[0]
income = income.groupby('education').agg(income_agg).reset_index()
income

,education,age,workclass,fnlwgt,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country
0,Academic degree,40.906770,Private,186427.490515,13.504990,Married-civ-spouse,Prof-specialty,Husband,White,Male,2783.699811,148.066799,43.297761,United-States
1,Higher education,36.332433,Private,188749.516427,10.361836,Married-civ-spouse,Adm-clerical,Husband,White,Male,586.416742,77.025953,39.454320,United-States
2,Lower secondary,47.083535,Private,209247.739709,3.298426,Married-civ-spouse,Other-service,Husband,White,Male,290.303874,61.390436,38.736077,United-States
3,Secondary / secondary special,38.135474,Private,190745.016961,8.436503,Married-civ-spouse,Craft-repair,Husband,White,Male,504.779882,64.238199,39.552563,United-States


In [24]:
app_record.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36457 entries, 0 to 36456
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID                   36457 non-null  int64  
 1   CODE_GENDER          36457 non-null  object 
 2   FLAG_OWN_CAR         36457 non-null  object 
 3   FLAG_OWN_REALTY      36457 non-null  object 
 4   CNT_CHILDREN         36457 non-null  int64  
 5   AMT_INCOME_TOTAL     36457 non-null  float64
 6   NAME_INCOME_TYPE     36457 non-null  object 
 7   NAME_EDUCATION_TYPE  36457 non-null  object 
 8   NAME_FAMILY_STATUS   36457 non-null  object 
 9   NAME_HOUSING_TYPE    36457 non-null  object 
 10  DAYS_BIRTH           36457 non-null  int64  
 11  DAYS_EMPLOYED        36457 non-null  int64  
 12  FLAG_MOBIL           36457 non-null  int64  
 13  FLAG_WORK_PHONE      36457 non-null  int64  
 14  FLAG_PHONE           36457 non-null  int64  
 15  FLAG_EMAIL           36457 non-null 